In [13]:
import torch
from datasets import load_dataset
from rouge import Rouge

import transformers

from transformers import PegasusForConditionalGeneration, PegasusTokenizer
import transformers
from trainer import Trainer
from torch.utils.data import DataLoader
from pl_bolts.optimizers.lr_scheduler import LinearWarmupCosineAnnealingLR
import wandb
from logger import log_metrics
import numpy as np

class PegasusCNNDatasetNRandom(torch.utils.data.Dataset):
    def __init__(self, model_name = 'google/pegasus-large', max_length=256, split = 'train', first_selection = 1):
        self.tokenizer = PegasusTokenizer.from_pretrained(model_name)
        self.tokenizer.max_length = max_length
        self.dataset = load_dataset('cnn_dailymail', '3.0.0', split = split)
        self.max_length = max_length
        self.first_selection = first_selection
        
        #we want to tokenize both our inputs and outputs before passing to the model
        #self.inputs = self.tokenizer(self.dataset['article'], max_length=self.max_length, truncation=True, padding="longest", return_tensors="pt")
        #self.outputs = self.tokenizer(self.dataset['highlights'], max_length=self.max_length, truncation=True, padding="longest", return_tensors="pt")

    def __len__(self):
        return len(self.dataset)

    def __getitem__(self, idx):
        text = self.dataset[idx]['article']
        text = text.split('. ')
        
        max_idx = max(1, len(text) - 3)
        final = text[:self.first_selection]
        final = final + text[np.random.randint(max_idx):]
        print(final)
        text = '. '.join(final)

        summary_text = self.dataset[idx]['highlights']
        return {'article_text':text, 'summary_text': summary_text}

dataset = PegasusCNNDatasetNRandom()
train_loader = DataLoader(dataset, batch_size=4, shuffle=True, num_workers=0)
for data in train_loader:
    break
#get dataloader


Found cached dataset cnn_dailymail (/home/da2986/.cache/huggingface/datasets/cnn_dailymail/3.0.0/3.0.0/1b3c71476f6d152c31c1730e83ccb08bcf23e348233f4fcc11e182248e6bf7de)


['(Health.com) -- Infants who undergo surgical procedures requiring general anesthesia in their first two years of life may be at increased risk of developing attention deficit hyperactivity disorder (ADHD) as they grow older', 'Warner, however, stresses that parents should not be unduly alarmed if their child requires general anesthesia', '"All we can say is that we can\'t exclude that this could be a problem," he says, noting that the findings do not prove cause and effect', "Very few young children undergo surgery requiring general anesthesia, and those who do typically have serious medical conditions that can't be ignored", 'Procedures might include hernia repair (to prevent a section of intestine from getting trapped and causing an obstruction), or surgeries to correct life-threatening abnormalities of the lungs or heart', 'Health.com: 9 food additives that may affect ADHD ', 'The new findings should not dissuade parents or doctors from approving necessary surgery, says Peter J', 

In [14]:
data

{'article_text': ['(Health.com) -- Infants who undergo surgical procedures requiring general anesthesia in their first two years of life may be at increased risk of developing attention deficit hyperactivity disorder (ADHD) as they grow older.Warner, however, stresses that parents should not be unduly alarmed if their child requires general anesthesia."All we can say is that we can\'t exclude that this could be a problem," he says, noting that the findings do not prove cause and effect.Very few young children undergo surgery requiring general anesthesia, and those who do typically have serious medical conditions that can\'t be ignored.Procedures might include hernia repair (to prevent a section of intestine from getting trapped and causing an obstruction), or surgeries to correct life-threatening abnormalities of the lungs or heart.Health.com: 9 food additives that may affect ADHD .The new findings should not dissuade parents or doctors from approving necessary surgery, says Peter J.Da